In [181]:
import pandas as pd 
import ast
from IPython.display import clear_output, display, HTML


combined_data = pd.read_csv(f'combined_data_remaining.csv', converters={'years_taught': ast.literal_eval, 'semester': ast.literal_eval,'course': ast.literal_eval, 'salaries': ast.literal_eval, 'departments': ast.literal_eval})
#combined_data = pd.read_csv(f'combined_data.csv')
salary_data = pd.read_csv(f'clean_salary_data.csv', converters={'years_taught': ast.literal_eval, 'salaries': ast.literal_eval, 'departments': ast.literal_eval})
new_combine = pd.read_csv(f'new_combined_data.csv', converters={'years_taught': ast.literal_eval, 'salaries': ast.literal_eval, 'departments': ast.literal_eval})


#left_only.head()
majors_df = pd.read_csv('code_departments.csv')
#print(majors_df.index[majors_df['Major'] == "CPMS"].to_list()[0])
#print(majors_df.at[majors_df.index[majors_df['Major'] == "CPMS"].to_list()[0], "Department"])
#print(majors_df.index[majors_df['Major'] == "FGSM"].to_list())

def find_similar(first, last, courses, outer, year):
    c = []
    for i, course in enumerate(courses):
        index = majors_df.index[majors_df['Major'] == course[:4]].to_list()
        if index != []:
            temp = majors_df.at[index[0], "Department"]
            if len(temp) == 4:
                c.append(temp)
            elif temp == 'Public Policy':
                c.append('PLCY')
    #print(c)
    names = []
    if len(first) > 1:
        names.append(first)
    if len(last) > 1:
        names.append(last)
    if "-" in last:
        names.append(last.split("-")[0])
        names.append(last.split("-")[1])

    outer["first_name"] = outer["name"].apply(lambda x: x.split(" ")[0])
    outer["last_name"] = outer["name"].apply(lambda x: x.split(" ")[1])

    df = outer.loc[(outer["name"]!=first+" "+last)&
                   ((outer["first_name"].isin(names)) |
                    (outer["real_name(s)"].apply(lambda x: any(substring in x for substring in names))) |
                    (outer["last_name"].isin(names))
                    )][["real_name(s)", "years_taught", "departments","salaries", "name"]]
    
    #print(df)
    df = df[df['departments'].apply(lambda x: any(dep in c for dep in x))]
    if len(df) >= 1:
        df = df.sort_values('real_name(s)', ascending=True)
    bad_dep = ["VPSA", "VPAF", "SVPA", "PRES", "VPAA", "LIBR", "VPA-", "DIT-", 
               "VPR-", "IT-N", "VPUR", "IT-I"]
    for i, row in df.iterrows():
        for dep in bad_dep:
            if dep in row["departments"]:
                df.drop(i, inplace = True)
                break
    return df.reset_index().drop("index", axis = 1)

responses = ["","n","never", "no", "nah", "nar", "nope", "poop"]

count = 0
'''combined_data["years_taught"] = combined_data["years_taught"].apply(lambda x: x.replace("'"," ")[1:-1].split(', '))
combined_data["salaries"] = combined_data["salaries"].apply(lambda x: x.replace("'"," ")[1:-1].split(', '))
combined_data["departments"] = combined_data["departments"].apply(lambda x: x.replace("'"," ")[1:-1].split(', '))'''

for i, row in combined_data.head(100).iterrows():
    semesters = row['semester']#.replace("'"," ")[1:-1].split(', ')
    missing = []
    for sem in semesters:
        year = int(sem[-5:])
        if "fall" in sem: 
            year += 1
        if year >= 2013 and year <= 2022 and (not year in row["years_taught"]) :
            if not year in missing:
                missing.append(year)
    if len(missing) >  0:
        first = row["name"].split(" ")[0]
        last = row["name"].split(" ")[1]
        #print(first, last)
        count +=1
        similar = find_similar(first,last,row["course"], salary_data, year)
        if len(similar) >= 1:
            years = row["years_taught"].copy()
            years.sort()
            missing.sort()
            print(f"years: {years}")
            print(f"missing years: {missing}")
            display(HTML(similar.to_html()))
            response = input(f"{first} {last}: ")
            if (not response in responses):
                new_years = similar.loc[int(response), 'years_taught']#[1:-1].replace(" "," ").split(",")
                combined_data.loc[i, 'years_taught'].extend(new_years)

                new_salaries = similar.loc[int(response), 'salaries']#[1:-1].replace(" "," ").split(",")
                combined_data.loc[i, 'salaries'].extend(new_salaries)

                new_departments = similar.loc[int(response), 'departments']#[1:-1].replace(" "," ").split(",")
                combined_data.loc[i, 'departments'].extend(new_departments)

                new_combine = new_combine.append(combined_data.loc[i], ignore_index=True)

            #row["years_taught"] = row["years_taught"].extend(similar.loc[int(response), "years_taught"])
            clear_output()
    combined_data.drop(i, inplace = True)

print(count)
combined_data.to_csv('combined_data_remaining.csv', index=False)
new_combine.to_csv('new_combined_data.csv', index = False)

0


In [184]:
df = pd.read_csv(f'combined_data.csv')
changed_names = pd.read_csv(f'new_combined_data.csv')

df = df.drop(df[df['name'] == "P KSHETRY"].index)
df = df.drop(df[df['name'] == "R APTER"].index)

for i, row in changed_names.iterrows():
    df = df.drop(df[df['name'] == row['name']].index)

result = pd.concat([df, changed_names], axis=0)
result = result.reset_index(drop=True)

result.to_csv('final_combine_data.csv', index = False)